\author{Authors: Lewis Li, Daniel Brito}
# Computing Wellbore and Surface Line Heat Losses


## Surface Heat Losses
We will calculate the steady state heat losses per year per 100ft as steam is injected through a pipe. We will consider 4 inch tubing in a N-80 pipe which is shown below in Figure 1. The insulation used is calcium silicate, and the thermal properties of both the pipe and insulation are shown in Table 1.

<figure>
<img src="../../figures/Pipe.png" width="400px"/>
  <figcaption>Figure 1. Schematic representation of idealized thermal resistances in buried surface lines (from [1]).</figcaption>
</figure>

|Symbol    | Units   | Value   | Description  |  
|---|---------|--------|------------------------------------------|
|$r_i$   | ft  | 0.1478  |  inner radius of pipe |  
|$r_o$   | ft  | 0.1667  |  outer radius of pipe |   
|$r_{ins}$   | ft  | 0.4167  | external radius of insulation |   
|$\lambda_p$   | Btu/ft-D-F  | 600  | thermal conductivity of pipe   |   
|$\lambda_{ins}$ | BTU/ft-D-degF  | 0.96  |thermal conductivity of insulation   |    
|$h_f$   | Btu/sq ft-D-F   | 48000  | film coefficient of heat transfer btw fluid inside the pipe and pipe wall   |   
|$h_{fc}$   | Btu/sq ft-D-F   | 154  |  film coefficient of heat transfer due to forced convection|   
|$h_{Pi}$   |  n/a | $\infty$ |  coefficient of heat transfer across scale deposits |   
|$h_{Po}$   | Btu/sq ft-D-F  | 48000  |  coeffcient of heat transfer btw pipe and insulation |   
|$T_b$   | F  | 550  |  steam temperature |   
|$T_a$   | F  | 60  |  ambient temperature |   
|$L$   | ft  | 100  |  length of pipe |   
|$t$   | days | 365  |  time period for evaluating losses |   


We will first specify the constants:

In [1]:
ri = 0.1478;     % ft
ro = 0.1667;     % ft
r_ins = 0.4167;  % ft
lamda_I = 0.96;  % Btu/ft-D-F 
lamda_P = 600;   % BTU/ft-D-degF
hf = 48000;      % Btu/sq ft-D-F
hfc = 154;       % Btu/sq ft-D-F
hpi = inf;       % Btu/sq ft-D-F
hpo = 48000;     % Btu/sq ft-D-F
Tb = 550;        % F
Ta = 60;         % F

### Without Insulation
In the first case, we assume that the pipe is not insulated. A 20 mph wind ($v_W$) is present normal to the pipe, therefore the conventive heat transform coefficient is estimated using the empirical relation from B.8 in (Prats,1985)
$$h_{fc}r_{ins} = 18v^{0.6}_W r_{ins}^0.6$$

In [2]:
v_w = 20;      
hfc = 18 * (v_w^0.6) * (ro^0.6) / ro;

For a bare pipe, the effective coefficient of heat transfer $h^{bare}_{Po}$ is estimated by the sum of 110Btu/sq ft-D-F from Table B.14 (Prats,1985) and the coefficient of heat transfer due to the forced convenction $h_{fc}$.

In [3]:
hrc = 110; % Btu/sq ft-D-F 
h_bare = hfc + hrc;  

Since there is no insulation, the specific heat resistance is evaluated as:
$$R_h = \frac{1}{2\pi}\left[ \frac{1}{h_fr_i} + \frac{1}{h_{pi}r_i} + \frac{1}{\lambda_p} \ln \frac{r_o}{r_i} + \frac{1}{h^{bare}_{po}r_o} \right]$$

In [4]:
R_h = (1 / (hf * ri) + 1 / (hpi * ri) + ...
(1 / lamda_P) * log(ro/ri) + 1 / ( (h_bare) * ro ))/(2*pi)

R_h =

    0.0029


To compute the heat loss we need to know the steam temperature and ambient temperature. We will assume the steam is $T_b$ 500F, and ambient $T_a$ is 60F. The heat loss per unit length is then:

$$Q_{loss} = \frac{T_b-T_a}{R_h}$$

In [5]:
Qls = (Tb - Ta) / R_h; %Btu/ft-D

For a pipe of length $L$ 100 ft over a period of 365 days $t$, the loss is:
$$Q_l = Q_{loss} L t$$

In [6]:
L = 100; % ft
t = 365; % days

Ql = Qls * L * t; % BTU

In [7]:
display('For an uninsulated pipe:');
display(['Specific thermal resistance was ' ...
num2str(R_h,3) ' BTU/ft-D'])
display(['Heat loss per unit length was ' ...
num2str(Qls,3) ' BTU/ft-D'])
display(['Heat loss from ' num2str(L)  ...
' of pipe over ' num2str(t) ' days was ' ...
num2str(Ql,3) ' BTU'])

For an uninsulated pipe:
Specific thermal resistance was 0.00293 BTU/ft-D
Heat loss per unit length was 1.67e+05 BTU/ft-D
Heat loss from 100 of pipe over 365 days was 6.11e+09 BTU


### With Insulation

When the pipe is insulated, we need to account for the heat loss across dart on outside of the pipe, across the insulation, and losses due to atmosphere from forced convenction. Thus, the expression for estimating specific heat resistance becomes:

$$R_h = \frac{1}{2\pi}\left[ \frac{1}{h_fr_i} + \frac{1}{h_{pi}r_i} + \frac{1}{\lambda_p} \ln \frac{r_o}{r_i} + \frac{1}{h_{po}r_o} + \frac{1}{\lambda_{ins}} \ln \frac{r_{ins}}{r_o} + \frac{1}{h_{fc}r_{ins}}\right]$$

In [8]:
R_h = (1 / (hf * ri) + 1 / (hpi * ri) + ...
(1 / lamda_P) * log(ro/ri) + 1 / ( (hpo) * ro ) + ...
(1 / lamda_I)*log(r_ins/ro) + 1 / (hfc * r_ins) )/(2*pi);

The heat loss per unit length is computed in the same manner as before:

In [9]:
Qls = (Tb - Ta) / R_h; %Btu/ft-D
Ql = Qls * L * t; % BTU

In [10]:
display('For an insulated pipe:');
display(['Specific thermal resistance was ' ...
num2str(R_h,3) ' BTU/ft-D'])
display(['Heat loss per unit length was '...
num2str(Qls,3) ' BTU/ft-D'])
display(['Heat loss from ' num2str(L)  ' of pipe over ' ...
num2str(t) ' days was ' num2str(Ql,3) ' BTU'])

For an insulated pipe:
Specific thermal resistance was 0.154 BTU/ft-D
Heat loss per unit length was 3.19e+03 BTU/ft-D
Heat loss from 100 of pipe over 365 days was 1.16e+08 BTU


## Wellbore Losses
We now look at the heat losses within a wellbore. Steam is injected into a well that is insulated with calcium silicate. The following properties are used in this analysis.

|Symbol    | Units   | Value   | Description  |  
|---|---|---|------------------------------|
|$T_b$   | F  | 600  |  steam temperature |   
|$T_a$   | F  | 100  |  ambient temperature in subsurface |   
|$t$   | days | 21  |  time period for evaluating losses | 
|$d$ | ft | 1000 | depth of well | 
|$r_o$   | ft  | 0.1458  |  outer radius of tubing |  
|$r_{ins}$   | ft  | 0.2292  |  insulation radius|  
|$r_{ci}$   | ft  | 0.3556  | casing inner radius | 
|$r_{co}$   | ft  | 0.401  | casing inner radius | 
|$r_w$ | ft | 0.5 | wellbore radius |
|$\alpha_{E}$ | sq-ft/D | 0.96 | thermal diffusivity of the earth |
|$\epsilon_{ins}$ | n/a | 0.9 | emissivity of insulation surface|
|$\epsilon_{ci}$ | n/a | 0.9 | emissivity of casing surface|
|$\lambda_E$   | Btu/ft-D-F  | 24  | thermal conductivity of earth   |   
|$\lambda_{cem}$   | Btu/ft-D-F  | 12  | thermal conductivity of cement   |   
|$\lambda_{ins}$   | Btu/ft-D-F  | 0.96  | thermal conductivity of insulation   |   
|$\lambda_{air}$   | Btu/ft-D-F  | 0.45  | thermal conductivity of air   |   
|$\nu_{air}$   | cP | 0.023  | air viscosity @ average annulus temperature   |   

The time function for the radiation boundary condition model is given in Prats, M. 1982 and is included with the data.


In [11]:
clear all;
% Temperature of steam (F)
Tb = 600;
% Ambient temperature of subsurface
Ta = 100;
% Time (days)
time = 21;

% Time function f(td) for the radiation boundary condition model
% Table 10.1 (Prats, M. 1982)
load('../../data/Table_10pt1_Prats.mat');

tableVertical = [0.1, 0.2, 0.5, 1.0, 2.0, ...
5.0, 10.0, 20.0, 50.0, 100.0];
tableHorizontal = [100.0, 50.0, 20.0, 10.0, ...
5.0, 2.0, 1.0, 0.5, 0.2, 0.1,...
0.05, 0.02, 0.01, 0.0];

% Well depth (ft)
depth = 1000;
% Tubing outer radius (ft)
ro = 0.1458;
% Insulation radius (ft)
rIns = 0.2292;
% Casing inner radius(ft)
rci = 0.3556;
% Casing outer diameter (ft)
rco = 0.401;
% Wellbore radius (ft)
rw = 0.5;
% Thermal diffusivity of the earth (sq-ft/D)
alphaE = 0.96;
% Emissivity of insulation surface
epsilonIns = 0.9;
% Emissivity of casing surface
epsilonCi = 0.9;
% Thermal conductivity of earth (Btu/ft-D-F)
lambdaE = 24;
% Thermal conductivity of cement (Btu/ft-D-F)
lambdaCem = 12;
% Thermal conductivity of insulation (Btu/ft-D-F)
lambdaIns = 0.96;
% Air viscosity @ average annulus temperature (Figure B.41)
viscosityAir = 0.023;
% Thermal conductivity of air (Btu/ft-D-F) (Figure B.72)
lambdaAir = 0.45;

The first step requires making an assumption such as the sum of all the thermal resistance is twice that due to the insulation.
$$R_h = \frac{2}{2\pi} \left[ \frac{ \ln(r_{ins}/r_0)}{\lambda_{ins}} \right]$$

In [12]:
RhGuess = (1/pi)*(log(rIns/ro))/lambdaIns;
Rprime = RhGuess

Rprime =

    0.1500


The next step is to evaluate $f(t_D)$ at 21 days. The dimensionalness time is evaluated using:
$$t_D = \frac{\alpha_Et}{r^2_w}$$
If this value is less than 100, we evaluate the value of $f(t_D)$ by interpolating Table 10.1 in Prats, else we evaluate the Ramey function
$$f(t_D) \approx \frac{1}{2} \ln(t_D) + 0.403$$

In [13]:
% Calculate dimensionless time
tD = alphaE*time/(rw*rw);
    
if (tD > 100.0)
    % Eqn 10.10 for tD < 100
    func = 0.5*log(tD) + 0.403;
else
    lookupVal = 2*pi*Rprime*lambdaE;
    % Interpolate table 10.1
    func = interp2(tableHorizontal, tableVertical,...
        Table_10pt1, lookupVal, tD);
end

We next evaluate the temperature at the outer radius of the annulus using Equation B.68 from Prats:
$$T_{ci} = T_A + \frac{T_b - T_A}{2\pi R_h} \left[ \frac{\ln(r_{co}/r_{ci})}{\lambda_P} + \frac{\ln(r_{w}/r_{co})}{\lambda_{com}} + \frac{\ln(r_{Ea}/r_{w})}{\lambda_{Ea}} + \frac{f(t_D)}{\lambda_E} \right]$$

In [14]:
% Step 3: Calculate Tci (temperature @ inner surface of casing)
% Eqn B.68 (neglecting 1st and 3rd terms in parenthesis)
Tci = Ta + ( (Tb - Ta)/(2*pi*Rprime) ) * ...
    (log(rw/rco)/lambdaCem + func/lambdaE);

We next calculate the temperature at the outer face of insulation using equation B.70 but only the 5th term.
$$T_{ins} = T_b + \frac{T_b - T_A}{2\pi R_h} \left[\frac{\ln r_{ins}/r_o}{\lambda_{ins}} \right]$$

In [15]:
% Step 4: Calculate Tins (temperature @ outer face of insulation)
% Eqn B.70 (only 5th term in parenthesis)
Tins = Tb - ((Tb - Ta)/(2*pi*Rprime))*...
    (log(rIns/ro)/lambdaIns);

The next step is to calculate the hRcAn coefficient of annular heat transfer due to radiation and convection. The average temperature in the annulus is hneeded to estimate air properties:

$$\overline{T_{an}} = 1/2 (T_{ins} + T_{ci}) $$
The air in the annulus:
$$\rho_a = 0.076 \left( \frac{460 + 60}{460 + \overline{T_{an}}} \right)$$

In [16]:
% Average temperature @ annulus
TavgAnnulus = 0.5*(Tins + Tci)
densityAir = 0.076*(460 + 60)/(460 + TavgAnnulus)

TavgAnnulus =

  258.6986


densityAir =

    0.0550


We next calculate the isobaric thermal coefficient of volume expansion for gas 
$$\beta_R = \frac{1}{T} + \frac{1}{z} \left( \frac{dz}{dT} \right)_p$$
(Assuming that air is an ideal gas.) The Grashof Number $N_{Gr}$ is needed to evaluate the effective thermal conductivity of the air:
$$N_{Gr} = 7.12e7\frac{(r_{ci} - r_{Ins})^3\rho_a^2\beta_G(T_{ins} - T_{ci})}{\nu_a^2}$$


In [17]:
% Isobaric thermal coefficient of volume 
% expansion for gas (Assuming that air is 
% an ideal gas.)
betaG = 1.0/(460 + TavgAnnulus);
% Eqn B.66: Grashof number
Grashof = 7.12e7*((rci - rIns)^3*densityAir^2*betaG*(Tins - ...
    Tci))/(viscosityAir*viscosityAir)

Grashof =

   2.0882e+05


Assuming a Prandtl number $(N_{Pr})$of 0.92, the apparent thermal conductivity of air in the annulus is evaluated as:

$$\lambda_{a,a} = 0.049 \lambda_{a}N_{Gr}^{0.333}N_{Pr}^{0.407}$$
The radiation temperature function $F(T_{ins},T_{ci})$ is evaluated as:
$$F(T_{ins},T_{ci}) = \left[ (T_{ins} + 460)^2 + (460 + T_{ci})^2 \right] (920+T_{ins} + T_{ci})$$
The coefficient of heat transfer by radiation and convection in the annulus is given by Willhite as:
$$h_{\zeta c,an} = 4.11\times 10^{-8} \left[ \frac{1}{\epsilon_{ins}} + \frac{r_{ins}}{r_{ci}} \left( \frac{1}{\epsilon_{ci}} - 1 \right) \right]^{-1} F(T_{ins},T_{ci}) + \frac{1}{r_e} \frac{\lambda_{a,a}}{ln(\frac{r_{ci}}{r_{ins}})}$$


In [18]:
% Prandtl number (assumption)
Prandtl = 0.92;
% Eqn B.65: Apparent thermal conductivity of air in the annulus
lambdaAAn = 0.049*lambdaAir*(Grashof^0.333)*(Prandtl^0.407)
% Eqn B.64: Radiation temperature function
F = ((460 + Tins)^2 + (460 + Tci)^2)*(920 + Tins + Tci)
% Eqn B.63: Coefficient of annular heat transfer
hRcAn = (4.11e-8 / (1/epsilonIns + (rIns/rci)*...
    (1/epsilonCi - 1)) ) * F+(1/rIns)*...
    lambdaAAn/log(rci/rIns)

lambdaAAn =

    1.2594


F =

   1.5089e+09


hRcAn =

   64.9439


Finally we compute $R_h$ using Equation 10.6:
\begin{eqnarray}
R_h = \frac{1}{2\pi} [ \frac{1}{h_fr_i}  + \frac{1}{h_{Pi}r_{i}} + \frac{\ln(r_o/r_i)}{\lambda_P} + \frac{1}{h_{Po}r_{o}} + \frac{\ln(r_{ins}/r_o)}{\lambda_{ins}} + \frac{1}{h_{\zeta c,an}r_{ins}} + \nonumber \\ + \frac{\ln(r_{co}/r_{ci})}{\lambda_{p}} + \frac{\ln(r_{w}/r_{co})}{\lambda_{cem}} + \frac{\ln(r_{Ea}/r_{w})}{\lambda_{Ea}} + \frac{f(t_D)}{\lambda_E} ] \nonumber 
\end{eqnarray}


In [19]:
% Step 6: Calculate Rh using Eqn 10.6
% Heat loss across insulation
Rh1 = log(rIns/ro) / lambdaIns;
% Heat loss through radiation and convection across annulus
Rh2 = 1.0 / (hRcAn * rIns );
% Heat loss across cement
Rh3 = log(rw/rco) / lambdaCem;
% Heat loss related to thermal resistance of earth
Rh4 = func / lambdaE;
% Eqn 10.6: Overall coefficient of heat loss
Rh = (1./(2*pi))* (Rh1 + Rh2 + Rh3 + Rh4)

Rh =

    0.1059


Since the initially assumed and calcuated values of $R_h$ in steps 1 and 6 do not agree, steps 2 to 6 are repeated until the difference between successive approximations of $R_h$ do not vary much.

In [20]:
RhGuess = (1/pi)*(log(rIns/ro))/lambdaIns;
Rprime = RhGuess;
error = Inf;
ConvergenceCriterion = 0.001;
NumIterations = 0;

% Check for convergence
while (error > ConvergenceCriterion)
    % Step 2: Calculate f(td) (Ramey function)
    % Calculate dimensionless time
    tD = alphaE*time/(rw*rw);
    
    if (tD > 100.0)
        % Eqn 10.10 for tD < 100
        func = 0.5*log(tD) + 0.403;
    else
        lookupVal = 2*pi*Rprime*lambdaE;
        % Interpolate table 10.1
        func = interp2(tableHorizontal, tableVertical,...
            Table_10pt1, lookupVal, tD);
    end

    % Step 3: Calculate Tci (temperature @ inner 
    % surface of casing) Eqn B.68 
    % (neglecting 1st and 3rd terms in parenthesis)
    Tci = Ta + ( (Tb - Ta)/(2*pi*Rprime) ) * ...
        (log(rw/rco)/lambdaCem + func/lambdaE);
    
    % Step 4: Calculate Tins (temperature @ outer 
    % face of insulation) Eqn B.70 
    % (only 5th term in parenthesis)
    Tins = Tb - ((Tb - Ta)/(2*pi*Rprime))*...
        (log(rIns/ro)/lambdaIns);
    
    %Step 5: Calculate hRcAn coefficient of annular 
    % heat transfer due to radiation and convection
    % Average temperature @ annulus
    TavgAnnulus = 0.5*(Tins + Tci);
    densityAir = 0.076*(460 + 60)/(460 + TavgAnnulus);
    
    % Isobaric thermal coefficient of volume 
    % expansion for gas (Assuming that air is 
    % an ideal gas.)
    betaG = 1.0/(460 + TavgAnnulus);
    % Eqn B.66: Grashof number
    Grashof = 7.12e7*((rci - rIns)^3*densityAir^2*betaG*(Tins - ...
        Tci))/(viscosityAir*viscosityAir);
    % Prandtl number (assumption)
    Prandtl = 0.92;
    % Eqn B.65: Apparent thermal conductivity of air 
    % in the annulus
    lambdaAAn = 0.049*lambdaAir*(Grashof^0.333)...
    *(Prandtl^0.407);
    % Eqn B.64: Radiation temperature function
    F = ((460 + Tins)^2 + (460 + Tci)^2)*(920 + Tins + Tci);
    % Eqn B.63: Coefficient of annular heat transfer
    hRcAn = (4.11e-8 / (1/epsilonIns + (rIns/rci)*...
        (1/epsilonCi - 1)) ) * F+(1/rIns)*...
        lambdaAAn/log(rci/rIns);
    
    % Step 6: Calculate Rh using Eqn 10.6
    % Heat loss across insulation
    Rh1 = log(rIns/ro) / lambdaIns;
    % Heat loss through radiation and convection 
    % across annulus
    Rh2 = 1.0 / (hRcAn * rIns );
    % Heat loss across cement
    Rh3 = log(rw/rco) / lambdaCem;
    % Heat loss related to thermal resistance of earth
    Rh4 = func / lambdaE;
    % Eqn 10.6: Overall coefficient of heat loss
    Rh = (1./(2*pi))* (Rh1 + Rh2 + Rh3 + Rh4);
 
    % Convergence check
    error = abs(Rh - Rprime);
    Rprime = Rh;
    
    NumIterations = NumIterations + 1;
end

display(['Took ' num2str(NumIterations) ...
' iterations for convergence']);


Took 3 iterations for convergence


The actual heat loss is evaluated as:
$$\dot{Q_{ls}} = (T_b - T_A)/R_h$$

The heat loss for the given depth of the well and given time period is:
$$\dot{Q_{t}} = \dot{Q_{ls}} d$$

In [21]:
%  Eqn 10.1: Heat loss per unit depth of the well (Btu/ft-D)
Qls = (Tb - Ta) / Rh;
% Heat loss for the given depth of the well 
% and the given time period
Qt = Qls * depth;

display(['Temperature of casing is ' num2str(Tci) ' degrees F']);
display(['Heat loss rate from ' num2str(depth) ...
    ' ft. deep well after ' num2str(time) ' days  is ' ...
    num2str(Qt,'%4.2e') ' BTU/Day']);

Temperature of casing is 193.3691 degrees F
Heat loss rate from 1000 ft. deep well after 21 days  is 4.62e+06 BTU/Day
